<a href="https://colab.research.google.com/github/wooohun/BERT-Summarizer/blob/main/BART_Abstractive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install dependencies
!pip install transformers==4.28.0
!pip install torch
!pip install datasets
!pip install evaluate

In [2]:
import pandas as pd
import torch
import nltk
import evaluate
from datasets import load_dataset, load_metric, DatasetDict
from evaluate import evaluator

In [3]:
# install kaggle
!pip install -q kaggle
!mkdir ~/.kaggle

# get kaggle api token from account -> API -> create new API Token
# move kaggle api token to kaggle folder
!cp -v kaggle.json ~/.kaggle

cp: cannot stat 'kaggle.json': No such file or directory


In [5]:
# download dataset
# !chmod 600 /root/.kaggle/kaggl
!kaggle datasets download -d gowrishankarp/newspaper-text-summarization-cnn-dailymail
!unzip newspaper-text-summarization-cnn-dailymail

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
unzip:  cannot find or open newspaper-text-summarization-cnn-dailymail, newspaper-text-summarization-cnn-dailymail.zip or newspaper-text-summarization-cnn-dailymail.ZIP.


In [4]:
# grab datasets
train = load_dataset("cnn_dailymail",'3.0.0', split="train[:34%]")
test = load_dataset("cnn_dailymail",'3.0.0', split="test[:51%]")
dataset = DatasetDict()
dataset['train'] = train
dataset['test'] = test
# formatted as:
# DatasetDict{
#   train: Dataset{ - len = 287113 * x%
#     features:[]
#     num_rows: int 
#   }
#   test: {} - len = 11490
#
# }



Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


In [5]:
from transformers import BartTokenizerFast, BartForConditionalGeneration

# using fast tokenizer
tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-base')
model  = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

In [6]:
def preprocessing(dataset):
  inputs = [article for article in dataset['article']]
  targets = [target for target in dataset['highlights']]
  tokenized_inputs = tokenizer(inputs, text_target=targets, max_length = 1024, truncation=True)

  # # set tokenizer to encode labels
  # with tokenizer.as_target_tokenizer():
  #   labels = tokenizer(dataset['highlights'], max_length = 128, truncation=True)
  return tokenized_inputs

In [7]:
# process all datasets in batches using fast tokenizer for efficiency
processed_dataset = dataset.map(preprocessing, batched = True)

Map:   0%|          | 0/97618 [00:00<?, ? examples/s]

Map:   0%|          | 0/5860 [00:00<?, ? examples/s]

In [ ]:
tokenizer.batch_decode(processed_dataset['train']['labels'][0], skip_special_tokens=True)

In [8]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

In [ ]:
!pip install --upgrade accelerate

In [10]:
# create training args
batch_size = 4
logging_steps = len(processed_dataset['train'])

args = Seq2SeqTrainingArguments(
    output_dir = "facebook-bart-base-finetuned-cnn-dailymail",
    learning_rate=5.6e-5,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    predict_with_generate=True,
    fp16=True,
    logging_steps=logging_steps
)

In [ ]:
# install metrics
!pip install bert_score
!pip install rouge_score

In [12]:
# load metrics
rouge = evaluate.load('rouge')
bert_score = evaluate.load('bertscore')

In [13]:
import numpy as np

# metric computation function to pass into trainer object
def metric_compute(predicted):
  predictions, labels = predicted
  # decode predictions, labels for metric computation
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  # batch_decode returns list of tokens, use nltk to convert list of tokens to list of sentences
  decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
  decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

  # # compute bert_score
  # bert_score_res = bert_score.compute(
  #     predictions = decoded_preds, references = decoded_labels, use_stemmer=True
  # )
  rouge_res = rouge.compute(
      predictions = decoded_preds, references = decoded_labels, use_stemmer=True
  )
  # result = {key: value.mid.fmeasure * 100 for key, value in bert_score_res.items()}
  # for key, value in rouge_res.items():
  #   result[key] = value.mid.fmeasure * 100
  result = {key: value.mid.fmeasure * 100 for key, value in rouge_res.items()}
  
  return {key: round(val, 4) for key, val in result.items()}

In [14]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [15]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=processed_dataset['train'],
    eval_dataset=processed_dataset['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=metric_compute
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


In [ ]:
!pip install pynvml nvidia-ml-py3

In [ ]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [ ]:
!pip install accelerate

In [ ]:
from accelerate import Accelerator
from torch.utils.data.dataloader import DataLoader

In [ ]:
training_ds = Dataloader(processed_dataset['train'], batch_size = )

dataloader = DataLoader(processed_dataset, batch_size=args.per_device_train_batch_size)

if args.gradient_checkpointing:
  model.gradient_checkpointing_enable()


accelerator = Accelerator(fp16=args.fp16)
model, optimizer, dataloader = accelerator.prepare(model, adam_bnb_optim, dataloader)

model.train()
for step, batch in enumerate(dataloader, start=1):
  loss = model(**batch).loss
  loss = loss / args.gradient_accumulation_steps
  accelerator.backward(loss)
  if step % args.gradient_accumulation_steps == 0:
    optimizer.step()
    optimizer.zero_grad()
